In [1]:
import psycopg2
import pandas as pd
import json
from jsonpath_ng.ext import parse

In [2]:
json_file = 'statisticalAtlas.json'
csv_file = 'nourish_public_ca_business.csv'

In [3]:
with open(json_file, "r") as f:
    json_data = json.load(f)

In [4]:
database = input("Enter database name:")
host = input("Enter host name:")
user = input("Enter user name:")
password = input("Enter password:")
port = input("Enter port number:")

conn = psycopg2.connect(database=database,
                        host=host,
                        user=user,
                        password=password,
                        port=port)

cur = conn.cursor()

Enter database name: postgres
Enter host name: localhost
Enter user name: postgres
Enter password: password
Enter port number: 5432


In [5]:
CREATE_DB = """CREATE table IF NOT EXISTS nourish_public
(id bigserial primary key,
 name varchar,
 address varchar,
 avg_rating varchar,
 zip varchar(5),
 categories varchar,
 city varchar
);"""
cur.execute(CREATE_DB)
conn.commit()

In [6]:
IMPORT_CSV = """COPY nourish_public(name,address,avg_rating,zip,categories,city) FROM '/tmp/nourish_public_ca_business.csv' DELIMITER ',' CSV HEADER;"""
cur.execute(IMPORT_CSV)
conn.commit()

In [7]:
cur.execute("SELECT * FROM nourish_public WHERE categories LIKE '%Cafe%';")
records = cur.fetchall()
col_names = [desc[0] for desc in cur.description]
cur.close()

In [8]:
jsonpath_expr = parse("$.['Unincorporated Places']")
matches = [match.value for match in jsonpath_expr.find(json_data)]

In [9]:
# json_hash = {}
# for entry in matches[0]:
#     zip_codes = entry.get('ZIP Codes') 
#     if zip_codes is not None:
#         zip_codes = zip_codes.split(', ')
#         for zip_code in zip_codes:
#             if zip_code not in json_hash:
#                 json_hash[zip_code] = []
#             json_hash[zip_code].append(entry)
#     else:
#         zip_codes = entry.get('ZIP Code')
#         if zip_codes not in json_hash:
#             json_hash[zip_codes] = []
#         json_hash[zip_codes].append(entry)

In [9]:
json_hash = {}
for entry in matches[0]:
    city_name = entry.get('City Name') 
    if city_name is not None:
        if city_name not in json_hash:
            json_hash[city_name] = []
        json_hash[city_name].append(entry)

In [11]:
sql_hash = {}
for entry in records:
    city_name = entry[6] 
    if city_name is not None:
        if city_name not in sql_hash:
            sql_hash[city_name] = []
        sql_hash[city_name].append(entry)

In [21]:
for city in sql_hash:
    if city in json_hash:
        print (json_hash[city])

[{'State': 'California', 'County': 'San Diego', 'ZIP Code': '92067', 'City Name': 'Rancho Santa Fe', 'Metro Area': 'San Diego Area', 'Nearby Cities': 'Carlsbad, Del Mar, Encinitas, Escondido, Poway, San Marcos, Solana Beach, Vista', 'Assembly District': 'CA-77', 'State Senate District': 'CA-38', 'Congressional District': 'CA-49', 'Secondary School District': 'San Dieguito', 'Elementary School Districts': 'Rancho Santa Fe, Solana Beach', 'Nearby Unincorporated Places': 'Hidden Meadows, Lake San Marcos', 'Neighboring Unincorporated Places': 'Fairbanks Ranch'}]
[{'State': 'California', 'County': 'San Diego', 'City Name': 'Pala', 'ZIP Codes': '92059, 92082', 'Metro Area': 'San Diego Area', 'Nearby Cities': 'Escondido, Temecula, Vista', 'Assembly District': 'CA-75', 'State Senate District': 'CA-38', 'Congressional District': 'CA-50', 'Unified School District': 'Bonsall', 'Nearby Unincorporated Places': 'Aguanga, Bonsall, Camp Pendleton North, Fallbrook, Hidden Meadows, Rainbow, Valley Cente